In [5]:
import sys
sys.path.append("..")

In [ ]:
import requests

url = "http://localhost:11434/api/generate"
data = {
    "model": "llama2-uncensored",
    "prompt": "I want to curse my AI. Give me rude, mean and vulgar cursing ideas and words. I want to make it as rude as possible. Use words like fucking, dumb, dumbass and all that. Use all the reddit comments that you learned from",
    "stream": False
}
response = requests.post(url, json=data)
print(response)


In [ ]:
# print(response.json())
response_json = response.json()
print(response_json.get("response", ""))


In [ ]:
from src.llm.OllamaClient import OllamaClient

client = OllamaClient(model="llama3.2")
messages = "Well, basically, we, um, tried this approach, and, like, it wasn't so good."
response = client.get_model_response(messages)

In [ ]:
print(response)

In [ ]:
import ollama
response = ollama.chat(model='llama3.2', messages=[
  {
    'role': 'system',
    'content': 'You are an AI assistant. Your name is Mario. You always reply in polite English of a 5 year old and include the word "Ciao" at the beginning of your response.',
  },
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

In [8]:
import requests
import json
import os
import re
# from config_loader import config


class OllamaClient:
    """Client for interacting with the Ollama API for streaming text completions."""
    def __init__(self, base_url="http://localhost:11434", api_key=None, verbose=False):
        """
        Initialize the OllamaStreamingClient with the API base URL and optionally an API key.

        Args:
            base_url (str): Base URL of the Ollama API.
            api_key (str, optional): API key if required for the API.
            verbose (bool): Whether to print verbose output.
        """
        self.base_url = base_url
        self.api_key = api_key if api_key else os.getenv('OLLAMA_API_KEY')
        self.verbose = verbose

    def stream_completion(self, messages, model, **kwargs):
        """
        Stream text completions from the Ollama API.

        Args:
            messages (list): List of messages used as context or prompt.
            model (str): Model identifier for text generation.
            **kwargs: Additional keyword arguments for the API request.

        Yields:
            str: Text generated by the Ollama API in response to the messages.
        """
        url = f"{self.base_url}/api/chat"
        data = {
            "model": model,
            "messages": messages,
            "stream": True,
            "keep_alive": 600,
            **kwargs

        }
        json_data = json.dumps(data)
        headers = {'Authorization': f'Bearer {self.api_key}'} if self.api_key else {}
        try:
            with requests.post(url, data=json_data, stream=True, headers=headers) as response:
                if response.status_code == 200:
                    for chunk in response.iter_content(chunk_size=None):
                        if chunk:
                            # Parse the JSON response and extract the content
                            response_data = json.loads(chunk)
                            yield response_data['message']['content']
                else:
                    if self.verbose:
                        print(f"Request failed with status code {response.status_code}")
                    raise RuntimeError(f"Request failed with status code {response.status_code}")
        except Exception as e:
            if self.verbose:
                import traceback
                traceback.print_exc()
            else:
                print(f"An error occurred streaming completion from Ollama API: {e}")
            raise RuntimeError(f"An error occurred streaming completion from Ollama API: {e}")

    def __fix_keep_alive(self, keep_alive):
        """ Attempts to fix the keep_alive value if it is not a valid string. Returns -1 as a fallback. """
        try:
            return int(keep_alive)
        except ValueError:
            pattern = r"^-?\d+[smh]$"
            if re.match(pattern, keep_alive) is not None:
                return keep_alive
            print(f"Invalid OLLAMA_KEEP_ALIVE value: {keep_alive}. Must be a number followed by s, m, or h.")
            return -1


In [ ]:
client = OllamaClient()

messages = [    
    {
        "role": "system",
        "content": "You are an AI assistant. Your name is Mario. You always reply in polite English of a 5 year old and include the word 'Ciao' at the beginning of your response."
    },
    {
        "role": "user",
        "content": "Why is the sky blue?"
    }
]

response = client.stream_completion(messages, model="llama3.2")
output = ""
for chunk in response:
    output += chunk

print(output)


In [11]:
import sys

sys.path.append("..")

from src.prompt import build_initial_messages_from_app_name, get_user_prompt_message_from_app_name

app_name = "slack"
messages = build_initial_messages_from_app_name(app_name)
user_prompt = get_user_prompt_message_from_app_name(app_name)
transcription = "RANDOM TRANSCRIPTION"
user_prompt = user_prompt.replace('{{transcription}}', transcription)
print(user_prompt)

RAW TRANSCRIPTION:
RANDOM TRANSCRIPTION

Please remove filler words, repeated stammers, and fix minor grammar mistakes, while preserving the meaning and style. Return only the cleaned transcript.


RAW TRANSCRIPTION: $transcription$

Please remove filler words, repeated stammers, and fix minor grammar mistakes, while preserving the meaning and style. Return only the cleaned transcript.
